In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1]) 
resnet.eval()


def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)  


def extract_features(image_path):
    image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        features = resnet(image_tensor)
    return features.squeeze().numpy().flatten()

csv_path = "lost_items_dataset.csv"  
df = pd.read_csv(csv_path)

df['features'] = df['image_path'].apply(lambda x: extract_features(x) if os.path.exists(x) else None)
df.dropna(inplace=True)

def find_best_match(found_image_path):
    found_features = extract_features(found_image_path)
    stored_features = np.stack(df['features'].values)
    similarities = cosine_similarity([found_features], stored_features)
    best_match_idx = np.argmax(similarities)
    return df.iloc[best_match_idx]

found_item_path = "test_image.jpg"  
best_match = find_best_match(found_item_path)

print("\nBest Match Found:\n")
print(f"Item Name      : {best_match['item_name']}")
print(f"Description    : {best_match['description']}")
print(f"Location       : {best_match['location']}")
print(f"Owner Contact  : {best_match['owner_contact']}")


C:\Users\AAYUSH THE GREAT\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AAYUSH THE GREAT\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Best Match Found:

Item Name      : Watch
Description    : Silver Rolex watch with a blue dial
Location       : 5th Avenue
Owner Contact  : owner5@example.com


In [1]:
import pandas as pd

# Load the CSV file
csv_path = "lost_items_dataset.csv"
df = pd.read_csv(csv_path)

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Strip spaces from all string values in the DataFrame
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Save the cleaned CSV file
cleaned_csv_path = "cleaned_lost_items_dataset.csv"
df.to_csv(cleaned_csv_path, index=False)

print(f"Cleaned CSV saved as {cleaned_csv_path}")


Cleaned CSV saved as cleaned_lost_items_dataset.csv


C:\Users\AAYUSH THE GREAT\AppData\Local\Temp\ipykernel_3596\157525900.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
